In [18]:
import os
import pandas as pd 
import numpy as np
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
data = pd.read_csv("C:\\Users\\norma\\OneDrive\\Desktop\\NOVICHOK\\novichok_356_cross_validation\\data\\csv\\dataset_no_imaginary.csv",delimiter=",",encoding="utf-8")

In [20]:
# Создал функцию, которая проходит по директории и читает наши '.xyz'-файлы
# Название каждого файла в моей директории отражено в датасете в столбце 'name' без расширения .xyz 
# Сконвертировал координаты в numpy-массив и закинул в новый столбец 'coordinates' в датасете

coordinates_list = []

def read_xyz_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()[2:]  
        coordinates = []
        for line in lines:
            parts = line.split()
            if len(parts) == 4:
                coordinates.append([float(parts[1]), float(parts[2]), float(parts[3])])
        return np.array(coordinates)

for index, row in data.iterrows():
    file_name = str(row['name']) + ".xyz"
    file_path = os.path.join("C:\\Users\\norma\\OneDrive\\Desktop\\NOVICHOK\\novichok_356_cross_validation\\xyz", file_name)
    coordinates = read_xyz_file(file_path)
    coordinates_list.append(coordinates)
data['coordinates'] = coordinates_list

In [21]:
data

,name,hf_gibbs_free_energy_ev,hf_electronic_energy_ev,hf_entropy_ev,hf_enthalpy_ev,hf_dipole_moment,hf_homo_ev,hf_lumo_ev,dft_gibbs_free_energy_ev,dft_electronic_energy_ev,dft_entropy_ev,dft_enthalpy_ev,dft_dipole_moment,dft_homo_ev,dft_lumo_ev,coordinates
0,002-M-BA1-b,-27275.076351,-27281.888683,1.795316,-27273.281035,0.87236,-8.8130,4.0489,-27617.108248,-27623.000160,1.877779,-27615.230469,1.47466,-6.2925,-1.4983,"[[-3.6610385002265, 6.93533048376608, -0.58345..."
1,002-M-BA1-pipi,-27275.128802,-27281.952769,1.772166,-27273.356635,1.19181,-9.2580,4.4306,-27617.107555,-27623.063060,1.822257,-27615.285298,1.69901,-6.7665,-1.3131,"[[-3.24946263013423, 6.99473259895769, -1.4223..."
2,005-MEL-CA,-25599.449771,-25604.305425,1.564143,-25597.885628,0.93514,-8.9909,4.6443,-25917.872472,-25922.091294,1.641525,-25916.230947,0.85984,-6.4902,-0.8343,"[[-5.415595, -7.691943, 0.824725], [-4.323509,..."
3,006-MEL-UA,-29133.284441,-29138.979188,1.668533,-29131.615908,2.68196,-8.2371,3.9070,-29496.320185,-29501.224258,1.791889,-29494.528296,3.15832,-6.1902,-1.1975,"[[-5.251705, -7.658659, 1.031666], [-4.20755, ..."
4,007-MEL-CR,-22635.215921,-22641.289493,1.651416,-22633.564505,6.85797,-8.6600,5.1451,-22922.502813,-22927.782911,1.687934,-22920.814880,6.66567,-6.3708,-0.2151,"[[-5.271343, -7.699293, 0.886577], [-4.227247,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,dimer_006121,-19613.756532,-19622.869155,1.793542,-19611.962991,6.65332,-9.2198,6.4409,-19866.663508,-19874.318183,1.856166,-19864.807342,7.42245,-6.4218,0.1743,"[[1.30646491596164, -0.57524990278906, -1.3873..."
154,dimer_006125,-20685.601738,-20693.928747,1.763538,-20683.838200,6.26955,-9.4013,4.9169,-20947.601629,-20954.521584,1.884566,-20945.717064,3.73848,-6.6824,-1.1960,"[[0.72411350301662, -0.84318903552034, 1.47813..."
155,dimer_006130,-20747.583954,-20757.490597,1.733202,-20745.850752,7.75520,-9.8085,11.3969,-21011.664617,-21019.968290,1.833384,-21009.831233,6.57082,-6.5023,0.1869,"[[-0.1568812460469, 1.12137289084067, -0.50992..."
156,dimer_006131,-22685.503101,-22693.815819,1.715946,-22683.787155,7.83449,-9.9891,11.2032,-22967.819716,-22974.854229,1.795052,-22966.024664,6.23344,-6.7157,0.1428,"[[-0.06237487272243, 0.8137375865662, -0.11994..."


In [22]:
# Для обучения использовал все дескрипторы, посчитанные на HF-3c уровне + столбец 'coordinates'  
# Первые 56 строк в документе - реальный датасет

features = ['hf_gibbs_free_energy_ev', 'hf_electronic_energy_ev', 'hf_entropy_ev',
            'hf_enthalpy_ev', 'hf_dipole_moment', 'hf_homo_ev', 'hf_lumo_ev', 'coordinates']
target = 'dft_lumo_ev'

train_data = data.iloc[18:]  
test_data = data.iloc[:18]  

In [23]:
# Скопировал сеты, т.к. предупреждение 'SettingWithCopyWarning' постоянно выскакивало при сглаживанни

X_train = train_data[features].copy()
y_train = train_data[target].copy()
X_test = test_data[features].copy()
y_test = test_data[target].copy()

In [24]:
# Сглаживание numpy-массива с координатами

X_train['coordinates'] = X_train['coordinates'].apply(np.ravel)
X_test['coordinates'] = X_test['coordinates'].apply(np.ravel)

In [25]:
# Определил максимальную длину сглаженных массивов для каждого сета
# Эта информация пригодится для дополнения сглаженных массивов, чтобы они были одинаковой длины 
# Это нужно для совместимости массива с моделями из сайкит-лерн

max_length_train = X_train['coordinates'].apply(len).max()
max_length_test = X_test['coordinates'].apply(len).max()

In [26]:
X_train['coordinates'] = X_train['coordinates'].apply(lambda x: np.pad(x, (0, max_length_train - len(x)), mode='constant'))
X_test['coordinates'] = X_test['coordinates'].apply(lambda x: np.pad(x, (0, max_length_test - len(x)), mode='constant'))

In [27]:
# Конкатенирую полученные сглаженные массивы с остальными столбцами 

X_train_numeric = pd.concat([X_train.drop(columns='coordinates'),
                             pd.DataFrame(np.vstack(X_train['coordinates']), 
                                          columns=[f'coord_{i}' for i in range(max_length_train)], 
                                          index=X_train.index)],
                            axis=1)

X_test_numeric = pd.concat([X_test.drop(columns='coordinates'),
                            pd.DataFrame(np.vstack(X_test['coordinates']), 
                                         columns=[f'coord_{i}' for i in range(max_length_test)], 
                                         index=X_test.index)],
                            axis=1)

In [28]:
# Numpy-массив у тестовой выборки (реальный датасет) короче, чем у тренировочной
# Определил разницу в длинах массивов, а затем привел к общей длине
 
max_columns = max(X_train_numeric.shape[1], X_test_numeric.shape[1])
X_train_numeric = X_train_numeric.reindex(columns=X_train_numeric.columns.union(X_test_numeric.columns), fill_value=0)
X_test_numeric = X_test_numeric.reindex(columns=X_train_numeric.columns, fill_value=0)

In [29]:
# Нормализация тренировочного датасета
# Процедуру применил ко всем дескрипторам, кроме координат

scaler = StandardScaler()

coord_columns = [col for col in X_train_numeric.columns if col.startswith('coord_')]
non_coord_columns = [col for col in X_train_numeric.columns if col not in coord_columns]

X_train_non_coord_scaled = scaler.fit_transform(X_train_numeric[non_coord_columns])
X_train_numeric_scaled = pd.concat([pd.DataFrame(X_train_non_coord_scaled, columns=non_coord_columns, index=X_train_numeric.index),
                                    X_train_numeric[coord_columns]], axis=1)

X_test_non_coord_scaled = scaler.transform(X_test_numeric[non_coord_columns])
X_test_numeric_scaled = pd.concat([pd.DataFrame(X_test_non_coord_scaled, columns=non_coord_columns, index=X_test_numeric.index),
                                   X_test_numeric[coord_columns]], axis=1)

In [30]:
# Нормализация таргета

scaler_y = StandardScaler()

y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [31]:
# Задаю разные значения оптимайзеров, скоростей и функций активации для поиска наилучших комбинаций

optimizers = ['adam', 'sgd', 'rmsprop', 'adagrad']
activation_functions = ['relu', 'sigmoid', 'tanh', "softmax"]
learning_rates = [0.0001, 0.001, 0.01, 0.1]

In [32]:
# Цикл перебирает различные комбинации оптимизаторов и функций активации, обучает модель для каждой комбинации, и оценивает ее точность с помощью 4-х метрик
# Для каждой из 64 возможных комбинаций проводится 5-кратная кросс-валидация
# Лист "results" содержит результаты кросс-валидации

results = []

############### 5-кратная кросс-валидация ############### 
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in kf.split(X_train_numeric_scaled):
    X_train_cv, X_test_cv = X_train_numeric_scaled.iloc[train_index], X_train_numeric_scaled.iloc[test_index]
    y_train_cv, y_test_cv = y_train_scaled[train_index], y_train_scaled[test_index]

    for optimizer in optimizers:
        for activation_function in activation_functions:
            for learning_rate in learning_rates:
                print(f"Optimizer: {optimizer}, Activation Function: {activation_function}, Learning Rate: {learning_rate}")
                ############### Компилирую и обучаю SLP модель ############### 
                model = tf.keras.Sequential([tf.keras.layers.Dense(1, activation=activation_function)])
                model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
                history = model.fit(X_train_cv, y_train_cv, epochs=100, batch_size=32, validation_split=0.2, verbose=0)

                ############### Оцениваю модель ############### 
                y_pred_scaled = model.predict(X_test_cv)
                y_pred_inverse = scaler_y.inverse_transform(y_pred_scaled)
                y_test_inverse = scaler_y.inverse_transform(y_test_cv)

                mse = mean_squared_error(y_test_inverse, y_pred_inverse)
                rmse = np.sqrt(mse)
                r_squared = r2_score(y_test_inverse, y_pred_inverse)
                mae = mean_absolute_error(y_test_inverse, y_pred_inverse)

                print(f"  MSE: {mse:.4f}, RMSE: {rmse:.4f}, R^2: {r_squared:.4f}, MAE: {mae:.4f}")

                results.append({
                    'Optimizer': optimizer,
                    'Activation Function': activation_function,
                    'Learning Rate' : learning_rate,
                    'MSE': mse,
                    'RMSE': rmse,
                    'R^2': r_squared,
                    'MAE': mae
                })

Optimizer: adam, Activation Function: relu, Learning Rate: 0.0001
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
  MSE: 0.9672, RMSE: 0.9834, R^2: -0.0583, MAE: 0.7501
Optimizer: adam, Activation Function: relu, Learning Rate: 0.001
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
  MSE: 1.2627, RMSE: 1.1237, R^2: -0.3817, MAE: 0.7783
Optimizer: adam, Activation Function: relu, Learning Rate: 0.01
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
  MSE: 2.0887, RMSE: 1.4452, R^2: -1.2856, MAE: 0.9125
Optimizer: adam, Activation Function: relu, Learning Rate: 0.1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
  MSE: 1.7735, RMSE: 1.3317, R^2: -0.9407, MAE: 0.9179
Optimizer: adam, Activation Function: sigmoid, Learning Rate: 0.0001
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
  MSE: 0.7939, RMSE: 0.8910, R^2: 0.1313, MAE: 0.5983
Optimizer: adam, Activation Function: sigmoid, Learning Rate: 0.001
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
  MSE: 1.0393, RMSE: 1.0195, R^2: -0.1373, MAE: 0.6750
Optimizer: adam, Activation Function: sigmoid, L

In [33]:
# ТОП-3 комбинаций из 64 возможных с лучшими MAE после 5-кратной кросс-валидации

results_sorted = sorted(results, key=lambda x: x['MAE'])
print("\033[1m" + "Top 3 combinations for 'LUMO' descriptor:".center(60) + "\033[0m")
print()
for i, result in enumerate(results_sorted[:3], 1):
   print(f"{i}. Optimizer: {result['Optimizer']}, Activation Function: {result['Activation Function']}, Learning Rate: {result['Learning Rate']}, MAE: {result['MAE']:.4f}")

         Top 3 combinations for 'LUMO' descriptor:          

1. Optimizer: sgd, Activation Function: sigmoid, Learning Rate: 0.0001, MAE: 0.4175
2. Optimizer: rmsprop, Activation Function: sigmoid, Learning Rate: 0.0001, MAE: 0.4209
3. Optimizer: sgd, Activation Function: sigmoid, Learning Rate: 0.01, MAE: 0.4247


In [34]:
# Повторно обучаю модель, но теперь на полном тренировочном датасете
# Беру оптимальные значения гиперпараметров, найденные с помощью кросс-валидации

best_combination = results_sorted[0]
best_optimizer = best_combination['Optimizer']
best_activation_function = best_combination['Activation Function']
best_learning_rate = best_combination['Learning Rate']

In [53]:
# Обучаю модель с лучшими значениями гиперпараметров 
# Использую EarlyStopping для предотвращения переобучения

model = tf.keras.Sequential([tf.keras.layers.Dense(1, activation=best_activation_function)])

if best_optimizer == 'sgd':
    optimizer = tf.keras.optimizers.SGD(learning_rate=best_learning_rate)
elif best_optimizer == 'adam':
    optimizer = tf.keras.optimizers.Adam(learning_rate=best_learning_rate)
elif best_optimizer == 'rmsprop':
    optimizer = tf.keras.optimizers.RMSprop(learning_rate=best_learning_rate)
elif best_optimizer == 'adagrad':
    optimizer = tf.keras.optimizers.Adagrad(learning_rate=best_learning_rate)
else:
    raise ValueError("Invalid optimizer specified")

model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
early_stopping = EarlyStopping(monitor='val_loss', mode='min', restore_best_weights=True, verbose=1, patience=60)
history = model.fit(X_train_numeric_scaled, y_train_scaled, epochs=2000, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 2.0190 - mae: 1.0500 - val_loss: 0.6823 - val_mae: 0.5652
Epoch 2/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.8313 - mae: 0.9961 - val_loss: 0.6813 - val_mae: 0.5646
Epoch 3/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 2.1094 - mae: 1.0848 - val_loss: 0.6804 - val_mae: 0.5640
Epoch 4/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.8486 - mae: 0.9954 - val_loss: 0.6799 - val_mae: 0.5637
Epoch 5/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.7426 - mae: 0.9496 - val_loss: 0.6795 - val_mae: 0.5634
Epoch 6/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.8283 - mae: 0.9835 - val_loss: 0.6789 - val_mae: 0.5630
Epoch 7/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 1.8225 - mae: 0.9813 - val_loss: 0.6784 - val_mae: 0.5627
Epoch 8/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.6662 - mae: 0.9465 - val_loss: 0.6778 - val_mae: 0.5623
Epoch 9/2000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 1.796

In [54]:
# График потерь по эпохам 
# Визуализация предела итераций, после которых модель переобучается

stopped_epoch = early_stopping.stopped_epoch
restore_epoch = early_stopping.stopped_epoch - early_stopping.patience

plt.plot(history.history['val_loss'])
plt.title('Validation Loss', fontsize=14)
plt.xlabel('Epoch', fontsize=14)
plt.ylabel('Loss', fontsize=14)

plt.axvline(x=stopped_epoch, color='red', linestyle='--', label=f'Stopped Epoch: {stopped_epoch}')
plt.axvline(x=restore_epoch, color='blue', linestyle='--', label=f'Restored Epoch: {restore_epoch}')

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.legend(fontsize=12, loc='upper left', bbox_to_anchor=(0.09, 0.985))
plt.show()

In [55]:
# Оцениваю обученную модель на тестовой выборке
# Провел обратное преобразование предсказанных и тестовых значений таргета, которые ранее были нормализованы

y_pred_scaled = model.predict(X_test_numeric_scaled)

y_pred_inverse = scaler_y.inverse_transform(y_pred_scaled)
y_test_inverse = scaler_y.inverse_transform(y_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


In [56]:
# MSE, RMSE, R^2, MAE - метрики качества

mse = mean_squared_error(y_test_inverse, y_pred_inverse)
rmse = np.sqrt(mse)
r_squared = r2_score(y_test_inverse, y_pred_inverse)
mae = mean_absolute_error(y_test_inverse, y_pred_inverse)

print("Mean Squared Error (MSE) on test data:", mse)
print("Root Mean Squared Error (RMSE) on test data:", rmse)
print("R-squared on test data:", r_squared)
print("Mean absolute error: ", mae)

Mean Squared Error (MSE) on test data: 3.6135610099271736
Root Mean Squared Error (RMSE) on test data: 1.900936876891806
R-squared on test data: -23.4195351316743
Mean absolute error:  1.859869336648782


In [62]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=y_test_inverse.flatten(), y=y_pred_inverse.flatten())
#plt.plot([y_test_inverse.min(), y_test_inverse.max()], [y_test_inverse.min(), y_test_inverse.max()], 'k--', lw=2) 
plt.xlabel('Actual Values', fontsize=14)
plt.ylabel('Predicted Values', fontsize=14)
plt.title('Single Layer Perceptron\nActual LUMO vs. Predicted LUMO', fontsize=14)

# plt.text(0.028, 0.80, f'- MSE: {mse:.6f}\n- RMSE: {rmse:.6f}\n- R-squared: {r_squared:.6f}\n- MAE: {mae:.6f}', color='red', fontsize=14, transform=plt.gca().transAxes)
# f'Optimizer: {best_optimizer}\nAct. function: {best_activation_function}\nLearning rate: {best_learning_rate}', color='green', fontsize=14, ha='left', transform=plt.gca().transAxes)

plt.text(0.55, 0.1, f'- MSE: {mse:.6f}', color='red', fontsize=14, transform=plt.gca().transAxes)
plt.text(0.55, 0.15, f'- RMSE: {rmse:.6f}', color='red', fontsize=14, transform=plt.gca().transAxes)
plt.text(0.55, 0.2, f'- R-squared: {r_squared:.6f}', color='red', fontsize=14, transform=plt.gca().transAxes)
plt.text(0.55, 0.25, f'- MAE: {mae:.6f}', color='red', fontsize=14, transform=plt.gca().transAxes)
plt.text(0.57, 0.3, f'Optimizer: {best_optimizer}\nAct. function: {best_activation_function}\nLearning rate: {best_learning_rate}', color='green', fontsize=14, ha='left', transform=plt.gca().transAxes)

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()

In [ ]:
# ТОП-5 лучших предсказанных значений и величина абсолютной разницы

y_test_flattened = y_test_inverse.flatten()
y_pred_flattened = y_pred_inverse.flatten()

comparison_df = pd.DataFrame({'Actual LUMO': y_test_flattened, 'Predicted LUMO': y_pred_flattened})
comparison_df['Absolute Difference'] = abs(comparison_df['Actual LUMO'] - comparison_df['Predicted LUMO'])

best_results = comparison_df.sort_values(by='Absolute Difference').head(5)
print("\033[1m{:^55}\033[0m".format("SINGLE LAYER PERCEPTRON"))
print("{:^55}".format("TOP-5 Predicted Values\n"))
print(best_results.to_string(index=False))

                SINGLE LAYER PERCEPTRON                
                TOP-5 Predicted Values
                
 Actual LUMO  Predicted LUMO  Absolute Difference
     -0.2151        0.269570             0.484670
     -0.8343        0.259502             1.093802
     -1.1975        0.167465             1.364965
     -1.3353        0.136514             1.471814
     -1.5997        0.096524             1.696224
